In [82]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from datetime import datetime

**Setting up the request/token**

In [83]:
auth = requests.auth.HTTPBasicAuth('z6CjULgiRVNzWw', 'xrww0UgaKLHfu91kqxrhrCXkdn_09Q')
data = {'grant_type': 'password',
       'username': 'pos99247',
       'password': 'rok1476011908'}
headers = {'User-Agent': 'databot/0.01'}
res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
TOKEN = res.json()['access_token']
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

**Try requesting some posts**

In [109]:
res = requests.get("https://oauth.reddit.com/r/wallstreetbets",
                   headers=headers)

In [110]:
print(res.json())

{'kind': 'Listing', 'data': {'modhash': None, 'dist': 27, 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'wallstreetbets', 'selftext': 'Your daily trading discussion thread. Please keep the shitposting to a minimum. \n\n^Navigate ^WSB|^We ^recommend ^best ^daily ^DD\n:--|:--                                 \n**DD** | [All](https://reddit.com/r/wallstreetbets/search?sort=new&amp;restrict_sr=on&amp;q=flair%3ADD) / [**Best Daily**](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADD&amp;restrict_sr=on&amp;t=day) / [Best Weekly](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADD&amp;restrict_sr=on&amp;t=week)\n**Discussion** | [All](https://reddit.com/r/wallstreetbets/search?sort=new&amp;restrict_sr=on&amp;q=flair%3ADiscussion) / [**Best Daily**](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADiscussion&amp;restrict_sr=on&amp;t=day) / [Best Weekly](https://www.reddit.com/r/wallstreetbets/search?so

In [117]:
for post in res.json()['data']['children']:
    print(post['data']['title'])

What Are Your Moves Tomorrow, April 23, 2021
Scam Alert: Wallstreetbets Telegram / Crypto
GME sets shareholder proxy vote for shareholder meeting on June 9, 2021
All-in MVIS before it was cool
[LEAKED] WSB Top Minds Developing DD
From $160K to $3.2M: Current YOLOs on GENI, VIAC, CC
Stocks sink after report says Biden will propose higher capital gains tax on the wealthy. Fuck my PLTR calls RIP
Weekly AMC YOLO update - holding
MVIS YOLO to Saturn
See I’m smart I picked contracts 1 week after DFVs expiration so I can lose even more money 💪
MicroVision Q1 Results Moved Up to April 29th 2021
The Big Short 2.0: why bears may soon reconsider sucking their paws in hibernation, and climb out of their caves. The TA thesis built on the Elliot wave principle, market cycles, Fibonacci and moving averages, applied to SPY.
OCGN +43% today, +$57k Gain
SEC considering more transparent derivatives and short selling rules
What would you say you do here?
WSB discussion today
5k -&gt; $93 Loss Porn + Margi

In [116]:
res.json()['data']['children'][10]

{'kind': 't3',
 'data': {'approved_at_utc': None,
  'subreddit': 'wallstreetbets',
  'selftext': '',
  'author_fullname': 't2_jgnpw',
  'saved': False,
  'mod_reason_title': None,
  'gilded': 0,
  'clicked': False,
  'title': 'MicroVision Q1 Results Moved Up to April 29th 2021',
  'link_flair_richtext': [{'e': 'text', 't': 'News'}],
  'subreddit_name_prefixed': 'r/wallstreetbets',
  'hidden': False,
  'pwls': 7,
  'link_flair_css_class': 'news',
  'downs': 0,
  'top_awarded_type': None,
  'hide_score': False,
  'name': 't3_mwg3ul',
  'quarantine': False,
  'link_flair_text_color': 'light',
  'upvote_ratio': 0.84,
  'author_flair_background_color': None,
  'subreddit_type': 'public',
  'ups': 321,
  'total_awards_received': 0,
  'media_embed': {},
  'author_flair_template_id': None,
  'is_original_content': False,
  'user_reports': [],
  'secure_media': None,
  'is_reddit_media_domain': False,
  'is_meta': False,
  'category': None,
  'secure_media_embed': {},
  'link_flair_text': 'News

In [87]:
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = df.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        }, ignore_index=True)

    return df

In [96]:
data = pd.DataFrame()
params = {'limit': 100}

In [103]:
for i in range(6):
    # make request
    res = requests.get("https://oauth.reddit.com/r/wallstreetbets/new",
                       headers=headers,
                       params=params)

    # get dataframe from response
    new_df = df_from_response(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # create fullname
    fullname = row['kind'] + '_' + row['id']
    # add/update fullname in params
    params['after'] = fullname
    
    # append new_df to data
    data = data.append(new_df, ignore_index=True)

IndexError: single positional indexer is out-of-bounds

In [104]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564 entries, 0 to 563
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   created_utc           564 non-null    object 
 1   downs                 564 non-null    float64
 2   id                    564 non-null    object 
 3   kind                  564 non-null    object 
 4   link_flair_css_class  557 non-null    object 
 5   score                 564 non-null    float64
 6   selftext              564 non-null    object 
 7   subreddit             564 non-null    object 
 8   title                 564 non-null    object 
 9   ups                   564 non-null    float64
 10  upvote_ratio          564 non-null    float64
dtypes: float64(4), object(7)
memory usage: 48.6+ KB


In [101]:
df1 = data.copy()

In [102]:
df1['created_utc']

0      2021-04-22T14:11:07Z
1      2021-04-22T22:12:30Z
2      2021-04-22T22:08:31Z
3      2021-04-22T21:22:09Z
4      2021-04-22T20:52:33Z
               ...         
495    2021-04-14T09:21:21Z
496    2021-04-14T09:18:09Z
497    2021-04-14T09:09:33Z
498    2021-04-14T09:04:09Z
499    2021-04-14T09:00:30Z
Name: created_utc, Length: 500, dtype: object

In [122]:
data.iloc[562]

created_utc                                          2021-04-13T13:58:29Z
downs                                                                 0.0
id                                                                 mqbfms
kind                                                                   t3
link_flair_css_class                                                 mods
score                                                               940.0
selftext                WSB is all about making high risk, high reward...
subreddit                                                  wallstreetbets
title                    WSB loves Gain / Loss / YOLO posts. Rule Review.
ups                                                                 940.0
upvote_ratio                                                         0.95
Name: 562, dtype: object

In [119]:
data.iloc[563]

created_utc                       2021-04-13T13:50:10Z
downs                                              0.0
id                                              mqb9yw
kind                                                t3
link_flair_css_class                              yolo
score                                            153.0
selftext                                              
subreddit                               wallstreetbets
title                   PLTR YOLO - I like the stock 😄
ups                                              153.0
upvote_ratio                                      0.86
Name: 563, dtype: object

In [124]:
res = requests.get("https://oauth.reddit.com/r/wallstreetbets/new",
                       headers=headers,
                       params={'after': 't3_mqb9yw'})

In [125]:
res.json()

{'kind': 'Listing',
 'data': {'modhash': None,
  'dist': 0,
  'children': [],
  'after': None,
  'before': None}}

In [126]:
res = requests.get("https://oauth.reddit.com/r/wallstreetbets/new",
                       headers=headers,
                       params={'after': 't3_mqbfms'})
res.json()

{'kind': 'Listing',
 'data': {'modhash': None,
  'dist': 0,
  'children': [],
  'after': None,
  'before': None}}

In [6]:
from psaw import PushshiftAPI
import datetime
import pandas as pd

In [3]:
api = PushshiftAPI()

start_time = int(datetime.datetime(2021, 4, 22).timestamp())

submissions = api.search_submissions(after=start_time,
                                         subreddit='wallstreetbets',
                                         filter=['url', 'author', 'title', 'subreddit', 'score', 'selftext'])

for submission in submissions:
    print(submission.created_utc)
    print(submission.title)
    print(submission.url)

1619207114
#safemoon
https://www.reddit.com/r/wallstreetbets/comments/mx3g39/safemoon/
1619207074
Holding EOW Inovio FDs when the gov’t drops their funding. ~$200 in, ~$4000 out
https://v.redd.it/gamx88r58zu61
1619207069
$AR - Undervalued oil and natural gas company with earnings next week
https://www.reddit.com/r/wallstreetbets/comments/mx3fht/ar_undervalued_oil_and_natural_gas_company_with/
1619207055
Which one of you is about to lose 300K ??
https://i.redd.it/9pc96sv08zu61.png
1619207050
ASO (Academy Sports and Outdoors)
https://www.reddit.com/r/wallstreetbets/comments/mx3f9v/aso_academy_sports_and_outdoors/
1619207050
Best cryptocurrency to buy?
https://www.reddit.com/r/wallstreetbets/comments/mx3f9n/best_cryptocurrency_to_buy/
1619207048
Voo
https://www.reddit.com/r/wallstreetbets/comments/mx3f8r/voo/
1619206955
AMD to $100 a share!!! They about to start filling orders that haven’t been manufactured because of covid. Everyone buy up
https://www.reddit.com/r/wallstreetbets/comments

1619203816
23k lost and most of it was my friend's money
https://www.reddit.com/r/wallstreetbets/comments/mx29wm/23k_lost_and_most_of_it_was_my_friends_money/
1619203806
CMG Chipotle - anyone following this?
https://www.reddit.com/r/wallstreetbets/comments/mx29rj/cmg_chipotle_anyone_following_this/
1619203796
True?
https://i.redd.it/w2218eyoyyu61.jpg
1619203752
$OCGN has already given us the good ... are we going to $BNGO and multiplying again?
https://www.reddit.com/r/wallstreetbets/comments/mx291d/ocgn_has_already_given_us_the_good_are_we_going/
1619203751
“Ken Griffin, Citadel... He’s trying to hide some of his money” - wow, even Donald Trump hated him
https://v.redd.it/f1n8e9rfyyu61
1619203738
Turning my boomer gains into GME gains. Small, but hungry for moon cheese. 🌙
https://i.redd.it/v61eh5oiyyu61.jpg
1619203732
Does this seem fishy??
https://www.reddit.com/gallery/mx28s9
1619203659
$SKLZ
https://www.reddit.com/r/wallstreetbets/comments/mx27sw/sklz/
1619203654
Yolo in Burford be

1619199655
Why am I like this
https://www.reddit.com/r/wallstreetbets/comments/mwy5yx/why_am_i_like_this/?utm_source=ifttt
1619199635
PMCB...does anyone love it and believe in what they do? 🤪👍🌟
https://www.reddit.com/r/wallstreetbets/comments/mx0qge/pmcbdoes_anyone_love_it_and_believe_in_what_they/
1619199629
Educational but expensive lessons
https://www.reddit.com/r/wallstreetbets/comments/mx0qdo/educational_but_expensive_lessons/
1619199622
Any recommendations?
https://www.reddit.com/r/wallstreetbets/comments/mx0qa9/any_recommendations/
1619199620
Just got access to a Bloomberg Terminal
https://www.reddit.com/r/wallstreetbets/comments/mx0q9i/just_got_access_to_a_bloomberg_terminal/
1619199531
Could the squeeze happen while I’m at work one day and I’d miss it?
https://www.reddit.com/r/wallstreetbets/comments/mx0p49/could_the_squeeze_happen_while_im_at_work_one_day/
1619199486
New to Investing - I want the Stonks!
https://www.reddit.com/r/wallstreetbets/comments/mx0oi6/new_to_investing

1619196576
MVIS IS UP 40% WOOOOOO
https://i.redd.it/yy603538dyu61.jpg
1619196563
How to vote for Swissquote users (GME or any US stocks)
https://www.reddit.com/r/wallstreetbets/comments/mwzlsw/how_to_vote_for_swissquote_users_gme_or_any_us/
1619196523
MVIS bought around $11 and sold around $18 🤑
https://i.redd.it/31nibyg2dyu61.jpg
1619196484
MVIS up 40% since open
https://www.reddit.com/r/wallstreetbets/comments/mwzks7/mvis_up_40_since_open/
1619196424
Eua and fda is coming baby please fasten your seatbelt
https://i.redd.it/5kn840rrcyu61.jpg
1619196414
INO loss thanks to govt shorty wins today...
https://i.redd.it/n5o86rsqcyu61.jpg
1619196405
Why Am I Not Rich?
https://i.redd.it/x1u5u5upcyu61.jpg
1619196403
Please for the love of God keep MVIS healthy.
https://www.reddit.com/r/wallstreetbets/comments/mwzjku/please_for_the_love_of_god_keep_mvis_healthy/
1619196302
Where did interest in GME go?
https://www.reddit.com/r/wallstreetbets/comments/mwzi69/where_did_interest_in_gme_go/
16191962

1619193396
#amcparty is trending on Twitter
https://www.reddit.com/r/wallstreetbets/comments/mwyfe3/amcparty_is_trending_on_twitter/
1619193373
Safemoon token
https://www.reddit.com/r/wallstreetbets/comments/mwyf21/safemoon_token/
1619193350
7ft 20kg 2" Olympic Weight Lifting bar BRAND NEW 400Kg Rated And Includes Clips
https://www.ebay.co.uk/itm/7ft-20kg-2-Olympic-Weight-Lifting-bar-BRAND-NEW-400Kg-Rated-And-Includes-Clips-/393223457063?_trksid=p2349624.m46890.l49286
1619193319
Me all day thinking of how I almost bought MVIS at close yesterday, but didn't.
https://i.redd.it/f8l4ovjj3yu61.jpg
1619193300
$RIOT is in for another huge bounce up. All we need is more buyers. COMPLETELY OVERSOLD stock atm, and is waiting to clearly rocket. LETS GET THE SHOW ON THE ROAD LADIES AND GENTS💰🍻🔮💎🤫🍿🚀🚀🚀🚀🚀🚀
https://i.redd.it/6k2s8jjh3yu61.jpg
1619193284
Roaring Kitty + Ape = Hodling!
https://twitter.com/TheRoaringKitty/status/1383153220671922177?s=20
1619193239
Regard cryptos rn
https://www.reddit.com

...

1619103839
PFE to $40 porfavor I want more fake vaccines plz plz with sugar on top 🤑
https://www.reddit.com/r/wallstreetbets/comments/mw6r0t/pfe_to_40_porfavor_i_want_more_fake_vaccines_plz/
1619103838
Kronk Stonks
https://vm.tiktok.com/ZMexfDVgw/
1619103827
Paper Hands
https://www.reddit.com/r/wallstreetbets/comments/mw6qug/paper_hands/
1619103764
Thoughts on Nancy Pelosi buying $1 million in Roblox?
https://www.barrons.com/articles/nancy-pelosi-husband-bought-roblox-microsoft-stock-51618264201
1619103752
Leaked footage of hedgefunds shorting stocks after GME (Credit: Zach Star)
https://v.redd.it/5qa158q2pqu61
1619103680
Paper Hands
https://www.reddit.com/r/wallstreetbets/comments/mw6ove/paper_hands/
1619103672
SPWR is MOVING FAST on new BIDEN greenhouse gas proposals! Looks like it’s ready to run? Thoughts?
https://www.reddit.com/r/wallstreetbets/comments/mw6or3/spwr_is_moving_fast_on_new_biden_greenhouse_gas/
1619103665
Digibyte - has anyone invested in this alt coin like I have? I 

1619100765
I still belive in GME$
https://i.redd.it/383pcjxbgqu61.jpg
1619100727
9:40am April 22nd 2021. I will never accomplish anything better than this. Its only downhill from here
https://www.reddit.com/gallery/mw5myr
1619100632
WALL STREET BABY IS LIVE Look at this, the good thing is that they also made a bridged version on pancake swap, where liquidity is terribly low (only 15k), the price moves literally $ 100 per stake i think it's a good opportunity one good investor is enough for price to go high
https://i.redd.it/1grbqmjxfqu61.jpg
1619100630
#Safemoon on ZBG #buythedip
https://i.redd.it/2ct7sadxfqu61.jpg
1619100599
YOLO - Multiple trades - Super short term trades ( may close in minutes or hours) Learn biatches 🖕🖕🚀🚀
https://i.redd.it/v4rki63ufqu61.jpg
1619100581
Bngo time
https://www.reddit.com/r/wallstreetbets/comments/mw5l1d/bngo_time/
1619100565
9:40am April 22nd 2021. I will never accomplish anything better than this. Its only downhill from here
https://i.redd.it/iikbogiq

1619097313
Microvision, Inc. (MVIS) Stock Price, News, Quote &amp; History - Yahoo Finance
https://finance.yahoo.com/quote/MVIS/?guccounter=1&amp;guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&amp;guce_referrer_sig=AQAAAIwURjgJr8ybsMUL6Oqwfy2u2pPtCFhOn5JL03toPwqbdV8ePLWX_sHQfBUbujtwSFgYJhJAge8yheQZiRUA5EkHH290JvWbkZjoUJAJH172vfPF_Gb9lHeqZT4AdrJznwW-jHl5RUXWmpYJjF-2rDzX9pw30ZaqEnhZ5PGJgjW1
1619097306
full yolo with some meme stocks
https://i.redd.it/nx6ew0p06qu61.png
1619097288
GME vs MVIS
https://i.redd.it/0rngdls55qu61.jpg
1619097249
US: New weekly jobless claims unexpectedly plunged to a new pandemic-era low
https://i.redd.it/rzdrgedp5qu61.png
1619097193
Hello there
https://www.reddit.com/r/wallstreetbets/comments/mw4fzx/hello_there/
1619097183
full yolo
https://www.reddit.com/r/wallstreetbets/comments/mw4fvq/full_yolo/
1619097169
???
https://i.redd.it/f7rj3dqm5qu61.jpg
1619097113
Novan to Present Clinical Data at Annual Meeting of the Society for Investigative Dermatology | Novan, In

1619091087
My list this month. All down 💪🏻🤣
https://i.redd.it/f9avn3ujnpu61.jpg
1619091086
TSLA YOLO to mars
https://i.redd.it/m2j96us7npu61.png
1619091080
Loading up on MVIS so I can Load up more on AMC and GME
https://www.reddit.com/r/wallstreetbets/comments/mw2pe1/loading_up_on_mvis_so_i_can_load_up_more_on_amc/
1619091039
Watching Margin Call for the First time before Kenny get the Call this morning😂. Best feeling ever. We are watching History in the making. 🙌💎🙌. 🦍💪🦍🚀🚀🌚🚀🚀
https://www.reddit.com/r/Superstonk/comments/mw2o0v/watching_margin_call_for_the_first_time_before/?utm_medium=android_app&amp;utm_source=share
1619091023
Is your business ready for the second wave of Covid-19
https://www.reddit.com/r/wallstreetbets/comments/mw2ov9/is_your_business_ready_for_the_second_wave_of/
1619090878
70,000 retail investors bought Deliveroo IPO at 390 hedgies shorted and now at 230 time to get the troops and send it to the moon! #togetherstronger
https://www.ft.com/content/b8000511-c910-483f-

1619079051
Question regarding article in German finance-website
https://www.reddit.com/r/wallstreetbets/comments/mw02ou/question_regarding_article_in_german/
1619078996
DFV on the psycholigical effect of price targets
https://www.reddit.com/r/Superstonk/comments/mvva2e/dfv_on_1_importance_of_perceived_price_target/?utm_medium=android_app&amp;utm_source=share
1619078922
AMC Megathread for 4/22/21
https://www.reddit.com/r/wallstreetbets/comments/mw01q9/amc_megathread_for_42221/
1619078593
Can’t wait for this rocket to take off🚀🚨 #safemoon #binance
https://i.redd.it/ve4zediemou61.jpg
1619078542
How to stop losing all of your fuckin money tutorial
https://www.reddit.com/r/wallstreetbets/comments/mvzz1u/how_to_stop_losing_all_of_your_fuckin_money/
1619078522
Anyone Bullish on CVM?
https://www.reddit.com/r/wallstreetbets/comments/mvzyw7/anyone_bullish_on_cvm/
1619078351
AMC let’s go ape shit on these hedgie fucknuts !!!
https://i.redd.it/988m2dlolou61.jpg
1619078177
SEC considering more tran

In [ ]:
submitted_time = datetime.datetime.fromtimestamp(submission.created_utc).isoformat()

In [12]:
api = PushshiftAPI()

start_time = int(datetime.datetime(2021, 4, 22).timestamp())

submissions = api.search_submissions(after=start_time,
                                         subreddit='wallstreetbets',
                                         filter=['url', 'author', 'title', 'subreddit', 'score', 'selftext'],
                                    limit=10)

df3 = pd.DataFrame()

for submission in submissions:
    df3 = df3.append({
        'time' : datetime.datetime.fromtimestamp(submission.created_utc).isoformat(),
        'author':submission.author,
        'title': submission.title,
        'score': submission.score,
        'selftext':submission.selftext,
        'url': submission.url
    }, ignore_index =True)

print(df3)

                 author  score  \
0           jam4ever_75    1.0   
1          loveangelove    1.0   
2        RogueYorkshire    1.0   
3  Careful-Translator51    1.0   
4            codestocks    1.0   
5   Southern-Apple-5686    1.0   
6     EmbarrassedAd2640    1.0   
7            codestocks    1.0   
8           LikelyCelek    1.0   
9       Flimsy_Crab6015    1.0   

                                            selftext                 time  \
0                      I m skeptical about their IPO  2021-04-23T15:32:21   
1                                          [removed]  2021-04-23T15:32:02   
2                                                     2021-04-23T15:31:53   
3                                                     2021-04-23T15:31:27   
4                                                     2021-04-23T15:31:18   
5                                                     2021-04-23T15:28:47   
6                                                     2021-04-23T15:28:18   
7        

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2582 entries, 0 to 2581
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   time    2582 non-null   object
 1   title   2582 non-null   object
 2   url     2582 non-null   object
dtypes: object(3)
memory usage: 60.6+ KB


In [11]:
start_time = int(datetime.datetime(2021, 4, 21).timestamp())

submissions = api.search_submissions(after=start_time,
                                         subreddit='wallstreetbets',
                                         filter=['url', 'author', 'title', 'subreddit'])
df2 = pd.DataFrame()

for submission in submissions:
    df2 = df2.append({
        'time' : datetime.datetime.fromtimestamp(submission.created_utc).isoformat(),
        'title': submission.title,
        'url': submission.url
    }, ignore_index =True)

print(df2)

                     time                                              title  \
0     2021-04-23T14:15:26                                   Miss MVIS LFG!!!   
1     2021-04-23T14:12:57                                   Miss MVIS LFG!!!   
2     2021-04-23T14:12:07  MindMed will uplist on the NASDAQ next Tuesday...   
3     2021-04-23T14:11:51                             Market at close - 4/23   
4     2021-04-23T14:11:19                                       Think Binovi   
...                   ...                                                ...   
4386  2021-04-21T00:04:59  If your GME stock isn‘t worth 6 or 7 figures y...   
4387  2021-04-21T00:03:59                       You know evofem biosciences?   
4388  2021-04-21T00:03:25                                                WW3   
4389  2021-04-21T00:00:30                                       WHOS LOOKING   
4390  2021-04-21T00:00:04  No Superman will do the job ! Let’s go apes ! ...   

                                       

In [14]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   author    10 non-null     object 
 1   score     10 non-null     float64
 2   selftext  10 non-null     object 
 3   time      10 non-null     object 
 4   title     10 non-null     object 
 5   url       10 non-null     object 
dtypes: float64(1), object(5)
memory usage: 608.0+ bytes


In [13]:
import pymysql

In [17]:
connection = pymysql.connect(host = 'localhost',
                            user= 'root',
                            password = 'pos99247',
                            db = 'mydb')
cursor = connection.cursor()

In [18]:
df3.columns

Index(['author', 'score', 'selftext', 'time', 'title', 'url'], dtype='object')

In [28]:
connection = pymysql.connect(host = 'localhost',
                            user= 'root',
                            password = 'pos99247',
                            db = 'mydb')
cursor = connection.cursor()

start_time = int(datetime.datetime(2021, 1, 1).timestamp())

submissions = api.search_submissions(after=start_time,
                                         subreddit='wallstreetbets',
                                         filter=['url', 'author', 'title', 'subreddit', 'score', 'selftext'])


for submission in submissions:
    submitted_time = datetime.datetime.fromtimestamp(submission.created_utc).isoformat()
    try:
        cursor.execute("""
        INSERT INTO wallstreetbets (time, author, title, score, selftext, url)
        VALUES (%s, %s, %s, %s, %s, %s)
        """, (submitted_time, submission.author, submission.title, submission.score, submission.selftext, submission.url))
        
        connection.commit()
    except Exception as e:
        print(e)
        connection.rollback()

(1062, "Duplicate entry 'https://i.redd.it/o390enmvrkv61.jpg' for key 'wallstreetbets.PRIMARY'")
'submission' object has no attribute 'selftext'
(1062, "Duplicate entry 'https://i.redd.it/fmbw65sukds41.jpg' for key 'wallstreetbets.PRIMARY'")
(1406, "Data too long for column 'url' at row 1")
'submission' object has no attribute 'selftext'
(1062, "Duplicate entry 'https://vm.tiktok.com/ZMexyMBBv/' for key 'wallstreetbets.PRIMARY'")
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
(1062, "Duplicate entry 'https://imgur.com/9ZbFEi7' for key 'wallstreetbets.PRIMARY'")
'submission' object has no attribute 'selftext'
(1062, "Duplicate entry 'https://i.redd.it/onn4

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
(1406, "Data too long for column 'url' at row 1")
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute '

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
(1406, "Data too long for column 'url' at row 1")
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute '

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'se

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'se

...

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'se

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
(1062, "Duplicate entry 'http://www.youtube.com/watch?v=POCHJAd3O…;feature=youtu.be' for

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
(1062, "Duplicate entry 'https://imgur.com/xUtq89b' for key 'wallstreetbets.PRIMARY'")
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'se

ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [33]:
connection = pymysql.connect(host = 'localhost',
                            user= 'root',
                            password = 'pos99247',
                            db = 'mydb')
cursor = connection.cursor()

start_time = int(datetime.datetime(2021, 1, 1).timestamp())

submissions = api.search_submissions(after=start_time,
                                     before=1612166400,
                                         subreddit='wallstreetbets',
                                         filter=['url', 'author', 'title', 'subreddit', 'score', 'selftext', 'created_utc'])


for submission in submissions:
    submitted_time = datetime.datetime.fromtimestamp(submission.created_utc).isoformat()
    try:
        cursor.execute("""
        INSERT INTO wallstreetbets (time, author, title, score, selftext, url)
        VALUES (%s, %s, %s, %s, %s, %s)
        """, (submitted_time, submission.author, submission.title, submission.score, submission.selftext, submission.url))
        
        connection.commit()
    except Exception as e:
        print(e)
        connection.rollback()

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'se

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'se

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'se

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'se

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'se

...

(1062, "Duplicate entry 'https://seekingalpha.com/news/3654070-koss-nears-sessions-high-a' for key 'wallstreetbets.PRIMARY'")
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
(1062, "Duplicate entry 'https://youtu.be/aZHTm0N59Rc' for key 'wallstreetbets.PRIMARY'")
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
(1062, "Duplicate entry 'https://youtu.be/qgQEBZpScIc' for key 'wallstreetbets.PRIMARY'")
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
(1062, "Duplicate entry 'HTTPS://wallstreetreddit.com' for key 'wallstreetbets.PRIMARY'")
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no a

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'se

'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'se

C:\Users\brian\.conda\envs\metis\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 524
  warnings.warn("Got non 200 code %s" % response.status_code)


'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
(1062, "Duplicate entry 'https://streamable.com/7ijcqn' for key 'wallstreetbets.PRIMARY'")
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftext'
'submission' object has no attribute 'selftex

In [34]:
int(datetime.datetime(2021, 4, 26).timestamp())

1619420400

In [ ]:
#start time should be the last fulldate in the DF prior to update
#end time shoudl be the present date at the time of the update
def datechanger(year, month, day):
    return time = int(datetime.datetime(year, month, day).timestamp())


def get_reddit_data(starttime, endtime):
    connection = pymysql.connect(host = 'localhost',
                            user= 'root',
                            password = 'pos99247',
                            db = 'mydb')
    cursor = connection.cursor()

    submissions = api.search_submissions(after=start_time,
                                         before=end_time,
                                         subreddit='wallstreetbets',
                                         filter=['url', 'author', 'title', 'subreddit', 'score', 'selftext', 'created_utc'])


    for submission in submissions:
        submitted_time = datetime.datetime.fromtimestamp(submission.created_utc).isoformat()
        try:
            cursor.execute("""
            INSERT INTO wallstreetbets (time, author, title, score, selftext, url)
            VALUES (%s, %s, %s, %s, %s, %s)
            """, (submitted_time, submission.author, submission.title, submission.score, submission.selftext, submission.url))
        
            connection.commit()
        except Exception as e:
            print(e)
            connection.rollback()